# Frozen randomness

Let's try to draw from nested logit model by drawing error terms. For logit models, this is simple because error terms are independent and therefore we can uniquely invert the CDF and simply draw from that. For nested models, this is not the case. However, we know we can write the probabilities as nested logits and therefore we think we can draw repeatedly like for logit models, taking the nesting structure into account.

Let's start with two levels and a model where we know the probabilities, i.e. we fix the utility functions and the nesting scales, like for red bus/blue bus. We then draw like ActivitySim does, and like we want to do.

Next, we extend to three levels, where the additional nest error term has not been derived yet.

In [1]:
import os
import sys

import numpy as np
from numpy.random import default_rng

In [2]:
def logsum(utilities, nest_scale=1.0):
    scaled_utils = utilities / nest_scale
    max_util = np.max(scaled_utils)
    return max_util + np.log(np.sum(np.exp(scaled_utils - max_util)))


# total probability of alternative being chosen is product of two terms:
# conditional probability of alternative given nest is chosen: exp(util / nest_scale)
# marginal probability that any alternative in nest is chosen: exp(nest_scale * logsum)
    
# If you think about a single case, the probabilities are indicator variables and we take the max of each. This is what Zenith does I think.
# Given that these expressions are those of two logits, it seems natural to draw correspondingly.
# This must be related to the strict derivation of max() distributions of Hunt.

# Two-level

In [3]:
alternatives = {1: "car", 2: "blue bus", 3: "red bus"}

utility_spec = {
    1: {"cost": -1.0, "asc": 0.0},
    2: {"cost": -1.0, "asc": 0.2},
    3: {"cost": -1.5, "asc": 0.1},
}

# blue and red bus are nested together with scale 0.5

def utility(x, utility_spec, alternative):
    assert alternative in utility_spec.keys()
    return utility_spec[alternative]["cost"] * x + utility_spec[alternative]["asc"]

In [4]:
cost = 3.0
nest_scale = 0.5

util_3 = utility(cost, utility_spec, 3)
util_2 = utility(cost, utility_spec, 2)
logsum_bus = logsum(np.array([util_2, util_3]), nest_scale=nest_scale)
nest_util = nest_scale * logsum_bus

util_1 = utility(cost, utility_spec, 1)
prob_1 = np.exp(util_1) / (np.exp(util_1) + np.exp(nest_util))

nest_cond_prob = np.exp(nest_util) / (np.exp(util_1) + np.exp(nest_util))
nest_marg_prob_2 = np.exp(util_2 / nest_scale) / (np.exp(util_2 / nest_scale) + np.exp(util_3 / nest_scale))
nest_marg_prob_3 = np.exp(util_3 / nest_scale) / (np.exp(util_2 / nest_scale) + np.exp(util_3 / nest_scale))

prob_2 = nest_cond_prob * nest_marg_prob_2
prob_3 = nest_cond_prob * nest_marg_prob_3

print(prob_1, prob_2, prob_3)
print(sum([prob_1, prob_2, prob_3]))

0.4452265282367507 0.5330453677531714 0.02172810401007798
1.0


## The ActivitySim way
Kind of pointless here but this is how we choose a single value - draw from U and pick whichever interval it falls into

In [5]:
%%time

probs = [prob_1, prob_2, prob_3]
cum_probs = [0] + list(np.cumsum(probs))

num_draws = 10000000

# now draw from U and put into arrays, then value count?
rng = default_rng(999)
rands = rng.uniform(size=num_draws)

hits, bins = np.histogram(rands, bins=cum_probs)
print(f"closed form: {probs},\nsimulated: {hits / num_draws}")

closed form: [0.4452265282367507, 0.5330453677531714, 0.02172810401007798],
simulated: [0.4450967 0.5331544 0.0217489]
CPU times: user 969 ms, sys: 78.1 ms, total: 1.05 s
Wall time: 1.14 s


In [6]:
def inverse_ev1_cdf(x, location=0.0, scale=1.0):
    "quantile function of EV1"
    #return location - (1.0 / scale) * np.log(-np.log(x))
    # let's follow https://en.wikipedia.org/wiki/Gumbel_distribution where the scale is proportional to variance (not variance^{-1})
    return location - scale * np.log(-np.log(x))

# for utilities with full set of ascs location=0. Do we always assume location=0 in estimation anyways?
# the scale of the error term is unidentified and therefore set to 1 in most applications, meaning the standard deviation is pi/sqrt(6)

## The Zenith way

Basically, probabilities are now indicators - choose at the lowest level, then walk up. Choice is product of these. We draw error terms for each alternative, where nest roots are now alternatives too.

OR: do we choose a

In [15]:
util_3 = utility(cost, utility_spec, 3)
util_2 = utility(cost, utility_spec, 2)
logsum_bus = logsum(np.array([util_2, util_3]), nest_scale=nest_scale)
util_1 = utility(cost, utility_spec, 1)

In [16]:
# conditional error term are given by logit with scale given by nest scale
num_draws_dec = 10000000
#mu = 1.0 / nest_scale

rng_dec = default_rng(9)
rands_dec = rng_dec.uniform(size = 2 * num_draws_dec)  # we need one for each alternative if num_draws_dec signifies the total number of choices we want to simulate
ev1_lower = inverse_ev1_cdf(rands_dec, scale=nest_scale)

lower_utils_2 = util_2 + ev1_lower[num_draws_dec:] 
lower_utils_3 = util_3 + ev1_lower[:num_draws_dec] 

#logsum_bus = logsum(np.array([lower_utils_2, lower_utils_3]), nest_scale=nest_scale)
ev1_upper = inverse_ev1_cdf(rng_dec.uniform(size=2*num_draws_dec))
nest_util = nest_scale * logsum_bus + ev1_upper[num_draws_dec:]

full_util_1 = util_1 + ev1_upper[:num_draws_dec]

choices = np.array([full_util_1, nest_util]).argmax(axis=0)
nest_indexes = np.nonzero(choices == 1)[0]
nest_choices = np.array([lower_utils_2[nest_indexes], lower_utils_3[nest_indexes]]).argmax(axis=0)
nest_choices += 1
choices = np.append(choices[choices == 0], nest_choices)

vals, counts = np.unique(choices, return_counts=True)
probs_dec = {i+1: counts[i] / num_draws_dec for i in vals}

print(f"closed form probs: {prob_1:.6f}, {prob_2:.6f}, {prob_3:.6f}")
print(f"  simulated probs: {probs_dec[1]}, {probs_dec[2]}, {probs_dec[3]}")

closed form probs: 0.445227, 0.533045, 0.021728
  simulated probs: 0.4452013, 0.5330709, 0.0217278


## The error term decomposition way -> not working. just use the zenith way and write it as indicators for individuals.

We can decompose the error term into one for the nest and one within nests according to Hildebrandt. However, I
cannot seem to reproduce the exact probabilities. Why?

Looks like one of the location parameters is wrong; 0.125 added to nest makes it right (tested for one set of params)

In [79]:
%%time
# conditional error term are given by logit with scale given by nest scale
num_draws_dec = 10000000

mu = 1.0 / nest_scale

rng_dec = default_rng(9)

rands_dec = rng_dec.uniform(size = 2 * num_draws_dec)  # we need one for each alternative if num_draws_dec signifies the total number of choices we want to simulate
ev1_dec = inverse_ev1_cdf(rands_dec, scale=1.0/mu)
lower_level_utils_2 = np.repeat(util_2, num_draws_dec) + ev1_dec[num_draws_dec:]
lower_level_utils_3 = np.repeat(util_3, num_draws_dec) + ev1_dec[:num_draws_dec]

#location_nest = - 1.0 / mu * np.log(2.0 * np.exp(mu * 0.5772))
location_nest = - np.log(2.0) / mu
#location_nest = (- np.log(2.0) / mu) - ((1.0 - 1.0 / (mu + 1.0)) * 0.57721 * mu / (mu**2 - 1.0))
#print(location_nest, - np.log(2.0) / mu)

scale_nest = mu / np.sqrt(mu**2 - 1.0)
nest_error_terms = inverse_ev1_cdf(rng_dec.uniform(size=num_draws_dec), location=location_nest, scale=1.0/scale_nest)

full_utils_2 = lower_level_utils_2 + nest_error_terms
full_utils_3 = lower_level_utils_3 + nest_error_terms

# what's the distribution of error term for car? it's a degenerate nest, so bottom level is 1
# this here agrees with Bhat and Koppelman's mode choice script.
full_utils_1 = util_1 + inverse_ev1_cdf(rng_dec.uniform(size=num_draws_dec))

choices = np.array([full_utils_1, full_utils_2, full_utils_3]).argmax(axis=0)
vals, counts = np.unique(choices, return_counts=True)
probs_dec = {i+1: counts[i] / num_draws_dec for i in vals}

print(f"closed form probs: {prob_1:.6f}, {prob_2:.6f}, {prob_3:.6f}")
print(f"  simulated probs: {probs_dec[1]}, {probs_dec[2]}, {probs_dec[3]}")
print(f"{prob_1 / probs_dec[1]:.4f}, {prob_2 / probs_dec[2]:.4f}, {prob_3 / probs_dec[3]:.4f}")

closed form probs: 0.445227, 0.533045, 0.021728
  simulated probs: 0.4755317, 0.5039483, 0.02052
0.9363, 1.0577, 1.0589
CPU times: user 4.12 s, sys: 2.3 s, total: 6.42 s
Wall time: 6.99 s


# Three-level

and recursive - maybe use Asim structure directly?


Could also use larch to apply models, would be great to add there too?